# Test loading Space ROSBAGS

In [ ]:
import os
import argparse
import cv2
import sys
import numpy as np
import rosbag
import rospy
from cv_bridge import CvBridge
from sensor_msgs.msg import Image
import bagpy
from bagpy import bagreader
import matplotlib.pyplot as plt
import pandas as pd

# Analyze rosbag

In [2]:
def imgmsg_to_cv2_(img_msg):
    dtype = np.dtype("uint8")  # Hardcode to 8 bits
    dtype = dtype.newbyteorder('>' if img_msg.is_bigendian else '<')
    image_opencv = np.frombuffer(img_msg.data, dtype=dtype).reshape((img_msg.height, img_msg.width, -1))
    return image_opencv


def imgmsg_to_cv2(img_msg):
    #if img_msg.encoding != "bgr8":
    #    rospy.logerr("This Coral detect node has been hardcoded to the 'bgr8' encoding.  Come change the code if you're actually trying to implement a new camera")
    dtype = np.dtype("uint8") # Hardcode to 8 bits...
    dtype = dtype.newbyteorder('>' if img_msg.is_bigendian else '<')
    # image_opencv = np.ndarray(shape=(img_msg.height, img_msg.width, 3), # and three channels of data. Since OpenCV works with bgr natively, we don't need to reorder the channels.
    #                 dtype=dtype, buffer=img_msg.data)
    try:
        image_opencv = np.ndarray(shape=(img_msg.height, img_msg.width, 3), dtype=dtype, buffer=img_msg.data)
    except:
        image_opencv = np.frombuffer(img_msg.data, dtype=dtype).reshape((img_msg.height, img_msg.width, -1))

    # If the byt order is different between the message and the system.
    if img_msg.is_bigendian == (sys.byteorder == 'little'):
        image_opencv = image_opencv.byteswap().newbyteorder()
    return image_opencv

def cv2_to_imgmsg(cv_image):
    img_msg = Image()
    img_msg.height = cv_image.shape[0]
    img_msg.width = cv_image.shape[1]
    img_msg.encoding = "bgr8"
    img_msg.is_bigendian = 0
    img_msg.data = cv_image.tostring()
    img_msg.step = len(img_msg.data) // img_msg.height # That double line is actually integer division, not a comment
    return img_msg

def extract_images_from_rosbag(bag_file, image_topic, output_dir):
    """Extract a folder of images from a rosbag.
    """
    bag = rosbag.Bag(bag_file, "r")
    count = 0
    for topic, msg, t in bag.read_messages(topics=[image_topic]):
        cv_img = imgmsg_to_cv2(img_msg=msg)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        cv2.imwrite(os.path.join(output_dir, "%06i.png" % count), cv_img)
        count += 1
    bag.close()
    return


def extract_events_from_rosbag(bag_file, ev_topic, output_dir):
    """Extract a folder of images from a rosbag.
    """
    bag = rosbag.Bag(bag_file, "r")
    count = 0
    for topic, msg, t in bag.read_messages(topics=[ev_topic]):
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        event_data = [(event.x, event.y, event.ts.to_nsec(), event.polarity) for event in msg.events]
        events = np.stack(event_data)

        dict(zip(['x','y','t','p'], events.T))
        sub_events = {k: v.cpu() for k, v in sub_events.items()}  
        np.savez(os.path.join(output_dir, "%010d.npz" % count), **sub_events)

        count += 1
    bag.close()
    return

In [3]:
dataset = "/data/storage/pellerito/StereoDavis"
segment = "/bin"
ros_bag_path = dataset + segment + ".bag"
dataset_output_dir = dataset + segment
output_dir_img = os.path.join(dataset_output_dir, "images_right")
output_dir_events = os.path.join(dataset_output_dir, "events_right")

In [4]:
b = bagreader(ros_bag_path)
print(b.topic_table)
velmsgs  = b.vel_data()

[INFO]  Data folder /data/storage/pellerito/StereoDavis/bin already exists. Not creating.
                    Topics                      Types  Message Count  \
0       /davis_left/events        dvs_msgs/EventArray            835   
1    /davis_left/image_raw          sensor_msgs/Image            556   
2          /davis_left/imu            sensor_msgs/Imu          27803   
3      /davis_right/events        dvs_msgs/EventArray            835   
4   /davis_right/image_raw          sensor_msgs/Image            557   
5         /davis_right/imu            sensor_msgs/Imu          27795   
6  /optitrack/davis_stereo  geometry_msgs/PoseStamped           5568   

    Frequency  
0   33.330981  
1   19.993155  
2  998.881638  
3   33.330981  
4   19.993155  
5  998.168491  
6  199.823916  


In [ ]:
extract_images_from_rosbag(bag_file=ros_bag_path, image_topic="/davis_right/image_raw", output_dir=output_dir_img)

In [ ]:
extract_events_from_rosbag(bag_file=ros_bag_path, ev_topic="/davis_left/events", output_dir=output_dir_events)

In [ ]:
true_poses = b.message_by_topic("/UAV0/true_pose")
veldf = pd.read_csv(true_poses)

In [ ]:

main_dir =  "/data/storage/pellerito/Marsyard/Datasets/"
ev_topic = "/camera/image_raw"

for segments in os.scandir(main_dir):
    segments_path = segments.path

    for scenes in os.scandir(segments_path):
        scenes_path = scenes.path
        if Path(scenes_path).suffix == ".bag":
            print(os.path.splitext(scenes_path)[0])
            output_folder = Path(os.path.splitext(scenes_path)[0]) / Path("images") 
            if not os.path.exists(os.path.splitext(scenes_path)[0]):
                os.mkdir(os.path.splitext(scenes_path)[0])
                os.mkdir(output_folder)
            try: 
                image_extractor_from_rosbag(bag_path=scenes_path,image_topic=ev_topic,output_folder=output_folder)
            except:
                print("\n error no images extracted maybe image_topic wrong?")
                b = bagreader(bag_path)
                print(b.topic_table)
            continue
        if Path(scenes_path).suffix == ".h5":
            continue 
        for files in os.scandir(scenes_path):
            files_path = files.path
            if Path(files_path).suffix == ".bag":
                print(os.path.splitext(files_path)[0])
                output_folder =  Path(os.path.splitext(files_path)[0]) / Path("images")
                if not os.path.exists(os.path.splitext(files_path)[0]):
                    os.mkdir(os.path.splitext(files_path)[0])
                    os.mkdir(output_folder)
                image_extractor_from_rosbag(bag_path=files_path,image_topic=ev_topic,output_folder=output_folder)
                continue 

# Import events from rosbag format with evlicious

In [34]:
import evlicious
from evlicious.io.rosbag_event_handle import RosbagEventHandle
from evlicious.io.h5_event_handle import H5EventHandle
from pathlib import Path
import cv2
import tqdm
import h5py
import numpy as np
import os
import yaml


accuracy = 1e9
rosbag_path = "/data/storage/pellerito/StereoDavis/bin.bag"
fps = 5

In [4]:
rosbag_handle = RosbagEventHandle.from_path(rosbag_path)

100%|██████████| 835/835 [02:48<00:00,  4.97it/s]


In [15]:
rosbag_handle.num_events

array([    4102,     6728,     8316,     9971,    11685,    13316,
          16769,    19302,    20950,    23566,    27997,    31956,
          37514,    43710,    48849,    55634,    64130,    71374,
          80092,    91033,   100500,   111381,   123585,   133466,
         143605,   155201,   164194,   173330,   184696,   195149,
         207689,   224434,   238154,   252598,   269169,   281986,
         295265,   312248,   326769,   342689,   362316,   376672,
         391299,   408408,   421236,   434021,   448423,   459515,
         470682,   483809,   494562,   506049,   520312,   532322,
         544630,   559660,   571600,   582823,   595564,   606277,
         618907,   634821,   647580,   660774,   677148,   689988,
         702908,   719692,   732647,   746833,   764389,   778463,
         792360,   808719,   821897,   834910,   850828,   864546,
         877431,   892374,   904093,   915396,   928866,   939417,
         950174,   962682,   972044,   981640,   995929,  1009

In [17]:
events = rosbag_handle.get_between_idx(0, rosbag_handle.num_events[-1])

In [21]:
h5_path = Path("/data/storage/pellerito/StereoDavis/bin/events.h5")

In [20]:
events.to(h5_path)

In [24]:
h5_handle = H5EventHandle.from_path(h5_path)

In [29]:
new_events = h5_handle.get_between_idx(0, 100)

In [ ]:
# load images with cv2 from a list of paths given the path of the dataset
def load_images_from_path(path):
    images = []
    for image_path in path:
        image = cv2.imread(image_path)
        images.append(image)
    return images

In [30]:
new_events.render(images)

array([ 52,  35,  58,   6,  94,   5, 143,  42, 211,  10,  20, 128,  35,
        58, 180,  89, 231, 184, 218,  70,  46, 109,  66,  58, 180, 182,
       202, 150, 185, 134, 176, 141, 209, 231,  22, 223,  88, 211, 112,
        34, 178, 108, 187, 226,  32, 143,  48,  10, 102, 222,  34, 186,
        85, 220, 160, 172, 204,  26, 178, 196, 185, 121, 220, 141, 219,
       116,   3,  48,  18, 127, 146, 198, 186,  56, 162,  28,  16, 142,
       124, 174, 222, 204, 109,  78, 196, 218, 207, 190,   1,   2, 178,
       192,  10,  54,  12, 126, 225,   1,   2,  68], dtype=uint16)

# Finally import poses from rosbag

In [7]:
import rosbag
import os
import rosbag
from sensor_msgs.msg import Image
import numpy as np
import cv2
import rosbag
from sensor_msgs.msg import Image
import numpy as np
import cv2
from pathlib import Path
from scipy.spatial.transform import Rotation
from scipy.spatial.transform import Slerp


rosbag_path = "/data/storage/pellerito/StereoDavis/bin.bag"
type = "geometry_msgs/PoseStamped"
topics = "/optitrack/davis_stereo"
output_dir = "/data/storage/pellerito/StereoDavis/bin/poses.txt"
bag = rosbag.Bag(rosbag_path, "r")

###



1518195889216209318
1518195889266226318
1518195889316243318
1518195889366260318
1518195889416277318
1518195889466295318
1518195889516312318
1518195889566329318
1518195889616346318
1518195889666364318
1518195889716381318
1518195889766398318
1518195889816415318
1518195889866433318
1518195889916450318
1518195889966467318
1518195890016484318
1518195890066501318
1518195890116519318
1518195890166536318
1518195890216553318
1518195890266570318
1518195890316588318
1518195890366605318
1518195890416622318
1518195890466639318
1518195890516657318
1518195890566674318
1518195890616691318
1518195890666708318
1518195890716726318
1518195890766743318
1518195890816760318
1518195890866777318
1518195890916794318
1518195890966812318
1518195891016829318
1518195891066846318
1518195891116863318
1518195891166881318
1518195891216898318
1518195891266915318
1518195891316932318
1518195891366950318
1518195891416967318
1518195891466984318
1518195891517001318
1518195891567019318
1518195891617036318
1518195891667053318


In [ ]:
def extract_poses(bag_path, topic):
    
    # get the entire file name from path without extension
    file_name = os.path.splitext(os.path.basename(rosbag_path))[0]
    #get the file path without the file name
    dirname = os.path.dirname(rosbag_path)
    sub_dataset_path = os.path.join(dirname, file_name)

    if not os.path.exists(sub_dataset_path):
        os.makedirs(sub_dataset_path)

    poses_path = os.path.join(sub_dataset_path, "poses.txt")
    timestamp_path = os.path.join(sub_dataset_path, "timestamps_poses.txt")
        
    bag = rosbag.Bag(bag_path, "r")
    for topic, msg, t in bag.read_messages(topics=[topic]):
        timestamp = t.to_nsec()
        x = msg.pose.position.x
        y = msg.pose.position.y
        z = msg.pose.position.z
        qx = msg.pose.orientation.x
        qy = msg.pose.orientation.y
        qz = msg.pose.orientation.z
        qw = msg.pose.orientation.w

        with open(poses_path, "a") as f:
            f.write(f"{x} {y} {z} {qx} {qy} {qz} {qw}\n")
        with open(timestamp_path, "a") as f:
            f.write(f"{timestamp}\n")

    bag.close()
    return


In [ ]:
def image_extractor_from_rosbag(bag_path,image_topic,output_folder):
    count = 0
    timestamps = []    
    bag = rosbag.Bag(bag_path, "r")

    for topic, msg, t in bag.read_messages(topics=[image_topic]):
        # Extract the image data from ROS message
        img_data = np.frombuffer(msg.data, dtype=np.uint8)

        try:
            img_shape = (msg.height, msg.width, 3) # Assuming color image
            cv_image = img_data.reshape(img_shape)
        except:
            img_shape = (msg.height, msg.width, 1) # Assuming grayscale image
            cv_image = img_data.reshape(img_shape)

        # Save the image to file
        #img_path = Path(output_folder) + str(t.to_nsec()) + ".png" # using timestamp as image name
        timestamps.append(t.to_nsec())
        img_path = os.path.join(output_folder, "%06i.png" % count)
        cv2.imwrite(img_path, cv_image)
        count += 1
    bag.close()

    tstamp_folder =  Path(os.path.splitext(bag_path)[0]) / Path("timestamps_images.txt")
    np.savetxt(tstamp_folder, np.array(timestamps))

In [ ]:
def interpolate_pose(pose1, pose2, tstamp1, tstamp2, target_time):
    alpha = (target_time - tstamp1) / (tstamp2 - tstamp1)

    x_before, y_before, z_before, qx_before, qy_before, qz_before, qw_before = pose1
    x_after, y_after, z_after, qx_after, qy_after, qz_after, qw_after = pose2

    # Perform linear interpolation for position (x, y, z)
    x_interpolated = x_before + alpha * (x_after - x_before)
    y_interpolated = y_before + alpha * (y_after - y_before)
    z_interpolated = z_before + alpha * (z_after - z_before)

    # Perform linear interpolation for quaternion (qx, qy, qz, qw)
    R_before = Rotation.from_quat([qx_before, qy_before, qz_before, qw_before]).as_matrix()
    R_after = Rotation.from_quat([qx_after, qy_after, qz_after, qw_after]).as_matrix()
    key_rots = Rotation.from_matrix(np.stack((R_before, R_after), axis=0))

    slerp = Slerp([tstamp1, tstamp2], key_rots)
    interp_rots = slerp(target_time)
    q_interpolated = interp_rots.as_quat()
    
    return (x_interpolated, y_interpolated, z_interpolated, *q_interpolated)


def interpolate_single_pose(poses, target_time, original_timestamps):
    index_before = np.searchsorted(original_timestamps, target_time) - 1
    index_after = index_before + 1

    # handle edge case of target time is 
    # after the last t or before the first t of original t stamps
    if index_after >= len(original_timestamps):
        return poses[index_before]
    if index_before < 0:
        return poses[index_after]

    return interpolate_pose(pose1=poses[index_before], 
                            pose2=poses[index_after], 
                            tstamp1=original_timestamps[index_before], 
                            tstamp2=original_timestamps[index_after], 
                            target_time=target_time
                            )

def interpolate_poses(poses, target_timestamps, original_timestamps):
    interpolated_trajectory = []
    for target_time in target_timestamps:
        interpolated_pose = interpolate_single_pose(poses, target_time, original_timestamps)
        interpolated_trajectory.append(interpolated_pose)
    return np.stack(interpolated_trajectory, axis=0)


def interpolate_depth(depth1, depth2, tstamps1, tstamps2, target_tstamp):
    alpha = (target_tstamp - tstamps1) / (tstamps2 - tstamps1)
    return alpha * depth1 + (1 - alpha) * depth2
